# Evaluating RL Algorithms for Autonomous Navigation in Webots

[Github Repository](https://github.com/grace-ortiz/COGS188_Final)

## Group members

- Katie Chung
- Jiawei Gao
- Grace Ortiz
- Hsiang-An Pao

# Abstract
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize:
- what your goal/problem is
- what the data used represents
- the solution/what you did
- major results you came up with (mention how results are measured)

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 3 to 5 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts.

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated.

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data

The agent generated its own data through its intereaction with the Webots environment. Each observation from the agent consisted of various sensor readings:

| Sensor | Data Description | Raw Data Type | 
|---|---|---|
| Camera | BGRA image frame | 1D byte array |
| GPS | XYZ coordinates | 3D float array |
| LiDAR | SICK LMS 291 point cloud distance | 1D float array | 
| Gyro | 3-axis angular velocity| 3D float array |

The raw input data was processed into useable obserations for the state space. Due to the length of the data processing functions implemented, the process can be found in the in webots_env.py<a name="processing"></a>[<sup>[1]</sup>](#processingnote).

The BGRA image frames were processed by removing the alpha channel and converting the HSV color space [Figure 1], creating a lane mask to detect the yellow center line [Figure 2]. A region of interest focuses on the bottom half of the frame and Canny edge detection is used to determine the lane edges. Finally, a sliding window algorithm tracks lane positionsby identifying lane pixels in successive windows. Originally we attempted to use a combined mask to detect both the solid yellow lane line and the dotted white lane lines. However, due to the resolution of the simulated camera in Webots, the white line proved too difficult to detect consistently and was removed from the lane mask. The center of the lane was calculated by determining a fixed pixel distance from the yellow center lane line rather than the center of the two lane lines. 

<img src="final_imgs/frame.png" alt="WeBots Car Camera Frame" width="400">
<p align="left"><em>Figure 1: WeBots car camera frame</em></p>
<img src="final_imgs/lane_mask.png" alt="WeBots Car Camera Lane Mask" width="400">
<p align="left"><em>Figure 2: Assocated lane mask for WeBots frame</em></p>


The GPS x and y coordinates were extracted and passed to the state space, and the speed was extracted from the GPS and converted from m/s to km/h. The LiDAR data is processed by replacing any null values with a safe maxium distance of 100.0 km. The minimum obstacle distance is extracted and passed to the state space. Using the LiDAR's field of view (FOV) and the index of the minimum distance, the angle of the nearest obstacle is calculated and converted from radians to degrees. The gyroscope data is not used in the state space, however the x value is extracted and used to detect collisions by determining if the agent has flipped over. 

In [ ]:
# action space: [steering, speed]
self.action_space = spaces.Box(
    low=np.array([MIN_STEER_ANGLE, MIN_SPEED]), 
    high=np.array([MAX_STEER_ANGLE, MAX_SPEED]),
    dtype=np.float32
)

# state space 
self.state_space = spaces.Dict({
    "speed": spaces.Box(low=0, high=MAX_SPEED, shape=(1,), dtype=np.float32),  # gps speed
    "gps": spaces.Box(low=-np.inf, high=np.inf, shape=(2,), dtype=np.float32),  # (x, y) gps coordinates
    "lidar_dist": spaces.Box(low=0, high=100, shape=(1,), dtype=np.float32),  # distance to nearest obstacle
    "lidar_angle": spaces.Box(low=-90, high=90, shape=(1,), dtype=np.float32),  # angle to nearest obstacle
    "lane_deviation": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),  # pixels away from lane center
    "lane_mask": spaces.Box(low=0, high=1, shape=(64, 128, 1), dtype=np.uint8)  # binary mask for lane line (yellow line only)
})

The state space had 6 total variables: `speed`, `gps`, `lidar_dist`, `lidar_angle`, `lane_deviation`, `lane_mask`. The number of obervations was dependent on the number of episodes the agent trained and how many steps each epsiode lasted. Webots by default runs at 32ms per time step, meaning approximately 31 observations will be recorded per 1 second of simulation time. The most critical values are `lane_deviation`, `speed`, and `lidar_dist` since these variabes are the most crucial in encouraging the agent to make foward progress and to reach the goal without crashing. All three of these variables are represented in the state space as a 1D float array. 


The action space is comprised of two variables: steering angle and speed, represented together as a 2D float array. The steering angle ranges from -0.5 to 0.5 radians to avoid over turning the wheels and causing joint disruptions within the simulation. The set steering angle is also smoothed, meaning that changes are capped to +/- 0.1 radians to allow for smooth turning. The speed ranges from 0 to 150 km/h, however to ensure that the cruising speed does not jump back and forth to the extremes, cruising speed changes are limited 75.0 km/h maximum. 

# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared.

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

main point: overall alg performance comparison
sub point 1: compare how different models handled the state space (continous vs discrete)
sub point 2: compare suitability for large vs small state spaces
sub point 3: compare convergence speed 
sub point 4: hyperparameter exploration


# Results

You may have done tons of work on this. Not all of it belongs here.

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Probably you should include a learning curve to demonstrate how much better the model gets as you increase the number of trials

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Generally reinforement learning tasks may require a huge amount of training, so extensive grid search is unlikely to be possible. However expoloring a few reasonable hyper-parameters may still be possible.  Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5

Maybe you do model selection again, but using a different kind of metric than before?  Or you compare a completely different approach/alogirhtm to the problem? Whatever, this stuff is just serving suggestions.



# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.


### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   


### Future work
Looking at the limitations and/or the toughest parts of the problem and/or the situations where the algorithm(s) did the worst... is there something you'd like to try to make these better.

### Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="processingnote"></a>1.[^](#processing): https://github.com/grace-ortiz/COGS188_Final/blob/main/webots/controllers/webots_env.py<br>

